In [1]:
# Importing dependencies
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

In [2]:
# Loading Data
transform =transforms.Compose([transforms.ToTensor()])

train_dataset=datasets.MNIST('data',train=True,download=True,transform=transform)
test_dataset=datasets.MNIST('data',train=False,download=True,transform=transform)
#transforms.ToTensor() - changing the images into a format - tensors 

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9912422/9912422 [00:01<00:00, 5093121.52it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28881/28881 [00:00<00:00, 333110.67it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1648877/1648877 [00:00<00:00, 3113098.44it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4542/4542 [00:00<00:00, 7450343.67it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [3]:
#print(train_dataset[0])

In [4]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [5]:
# Define the image classifier model
# class SimpleModel(nn.Module):
#     def __init__(self):
#         super(SimpleModel, self).__init__()
        
#        
#         self.fc1 = nn.Linear(28*28, 128)  # Flatten the 28x28 pixels into 1 row of 784 numbers
#         self.fc2 = nn.Linear(128, 10)     # Output 10 guesses (0-9 numbers)

#     def forward(self, x):
#         x = x.view(-1, 28*28)  # Flatten the image from 28x28 to 1 long line
#         x = torch.relu(self.fc1(x))  # Look at the first part (hidden layer)
#         x = self.fc2(x)  # Guess which number it is (final output)
#         return x

# Define the image classifier model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3),
            nn.ReLU()
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 22 * 22, 10)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

In [6]:


# build model
model = SimpleModel()

# Define the optimizer and loss function

criterion = nn.CrossEntropyLoss()


optimizer = optim.SGD(model.parameters(), lr=0.01)

# Train the model
epochs = 10
for epoch in range(epochs):
    for images, labels in train_loader:
        optimizer.zero_grad()  # Clear old mistakes
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


Epoch 1, Loss: 0.3751571774482727
Epoch 2, Loss: 0.2064821422100067
Epoch 3, Loss: 0.05533498525619507
Epoch 4, Loss: 0.1489965319633484
Epoch 5, Loss: 0.026863059028983116
Epoch 6, Loss: 0.01853412203490734
Epoch 7, Loss: 0.014898847788572311
Epoch 8, Loss: 0.2812590003013611
Epoch 9, Loss: 0.0191017035394907
Epoch 10, Loss: 0.03585870936512947


In [9]:

correct = 0
total = 0
with torch.no_grad():  # Don't do any training during testing
    for images, labels in test_loader:
        outputs = model(images)  
        _, predicted = torch.max(outputs, 1)  # The best guess
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')  #  how often the robot is it correct


Accuracy: 98.24%


In [11]:
# Save the trained model to a file
torch.save(model.state_dict(), 'model.pth')
